## Customize preprocess_data.py
---

## Learning Objectives

We fetched our own Swedish raw text data in `Lab2-1_acquiring_data.ipynb`, we learned how to find sentence boundary with custom functions in `Lab2-2_SentenceBoundary_and_Deduplicate.ipynb` , we also trained a GPTBPETokenizer and fitted it to our raw Swedish text with `Lab2-3_train_own_GPT2BPETokenizer.ipynb`. 

We are now ready to incorporate the custom sentence-splitter into preprocess_data.py and covert the raw Swedish text to, first json format, and then mmap format.

Therefore, the goal of this notebook is to integrate all knowledge gained from both Lab 1 as well as the above notebooks, and challenge ourselves to further customize the preprocess_data.py with a <a href="./Lab2-4_customize_process2mmap.ipynb#Custom-Sentence-Splitter">custom sentence-splitter</a>  function, and in the process, convert the new raw Sweden text to mmap format.

More specifically, this notebook will cover the steps to :

1.  Convert the extracted raw Swedish text from `webnyheter2013.txt` to `webnyheter2013.json`.
2.  Generate the mmap format files by default preprocess_data.py as the first step to ensure we have data necessary for the next notebook to run, in case time runs out.


Toward the end, there is a Mini-Challenge <a href="./Lab2-4_customize_process2mmap.ipynb#Mini-Challenge">Jump to view Mini-Challenge</a>.


1. Convert the extracted raw Swedish text from webnyheter2013.txt to webnyheter2013.json.

In [ ]:
!python create_loose_json.py --infile ../dataset/SV/webnyheter2013.txt --outfile ../dataset/SV/webnyheter2013.json

Below is the expected outputs :

        process 1000000 documents so far ...
        example:  – Vi har en bra generation som spelat tillsammans ett tag .

        finished processing 1249010 lines to loose json format

2. Generate the mmap format files by default preprocess_data.py as the first step to ensure we have data necessary for the next notebook to run, in case time runs out.

In [ ]:
INPUT_JSON_FILE='../dataset/SV/webnyheter2013.json'
OUTPUT_PATH='../dataset/SV/webnyheter2013_32kvocab'
VOCAB_FILE='../dataset/SV/32k/vocab.json'
MERGE_FILE='../dataset/SV/32k/merges.txt'
NUM_CPUS=16

In [ ]:
!python ./Megatron-LM/tools/preprocess_data.py \
                       --input $INPUT_JSON_FILE \
                       --output-prefix $OUTPUT_PATH \
                       --json-keys text \
                       --vocab-file $VOCAB_FILE \
                       --merge-file $MERGE_FILE \
                       --dataset-impl mmap \
                       --tokenizer-type GPT2BPETokenizer \
                       --workers $NUM_CPUS \
                       --append-eod

Below is the expected outputs :

    Processed 1248300 documents (52998.601302473544 docs/s, 5.869853647730749 MB/s).
    Processed 1248400 documents (53001.39142986273 docs/s, 5.870136451906283 MB/s).
    Processed 1248500 documents (53004.16423593737 docs/s, 5.870477584597603 MB/s).
    Processed 1248600 documents (53007.072626674184 docs/s, 5.870763528521501 MB/s).
    Processed 1248700 documents (53009.92668081499 docs/s, 5.871081674576178 MB/s).
    Processed 1248800 documents (53012.79399884911 docs/s, 5.871406835923378 MB/s).
    Processed 1248900 documents (53015.61341376629 docs/s, 5.8717617499445 MB/s).
    Processed 1249000 documents (53018.49277365899 docs/s, 5.8720826162486786 MB/s).

Now we get the default mmap files (xxx.bin and xxx.idx ) and therefore guarantee we have the data needed for the next notebook to run disregard whether we finish the mini-challenge or not. 

We can now move on. We start by copy the old preprocess_data.py and rename it to `MYpreprocess_data.py`. 

Note: As best practice, one never overwrites original python script existed in the given repo directly, one copies the original python script and rename it to a new python script, then work on the new python script, in case of irreversible failures, one can always refer to the original python script, and start again.

The below code block will duplicate the preprocess_data.py script and renamed the copied python script into a new python script called `MYpreprocess_data.py`.

In [ ]:
!cp ./Megatron-LM/tools/preprocess_data.py ./Megatron-LM/tools/MYpreprocess_data.py

<a id="Custom-Sentence-Splitter"></a>

The custom sentence-splitter `cut_sentence_with_quotation_marks` function is provided below for your convenience, please integrate this custom function into `MYpreprocess_data.py`.

In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize
def normal_cut_sentence(temp):
    return sent_tokenize(temp)

def cut_sentence_with_quotation_marks(text):
    p = re.compile("“.*?”")
    list = []
    index = 0
    length = len(text)
    for i in p.finditer(text):
        temp = ''
        start = i.start()
        end = i.end()
        for j in range(index, start):
            temp += text[j]
        if temp != '':
            temp_list = normal_cut_sentence(temp)
            list += temp_list
        temp = ''
        for k in range(start, end):
            temp += text[k]
        if temp != ' ':
            list.append(temp)
        index = end
    return list

<a id="Mini-Challenge"></a>

---
## **Mini-Challenge ** - integrate the custom sentence splitter into MYpreprocess_data.py

Task : Modify and overwrite `MYpreprocess_data.py` below to incoporate the custom `cut_sentence_with_quotation_marks`

Pass : Successfully run Mypreprocess_data.py with the custom sentence splitter cut_sentence_with_quotation_marks and generate the customSentenceSplit_text_document.bin and customSentenceSplit_text_document.idx files.

Note: the solution will be delivered to you at the end of Lab 2.

---
Modify the below cell block to overwrite `MYpreprocess_data.py`. 
After modification, Jump to Rerun cell to produce customSentenceSplit_text_document.bin and customSentenceSplit_text_document.idx files.
<a id="MODIFY_CELL"></a>
<a href="./Lab2-4_customize_process2mmap.ipynb#Rerun_Cell">Jump to ReRun Cell</a> 

In [ ]:
%%writefile ./Megatron-LM/tools/MYpreprocess_data.py 
# coding=utf-8
# Copyright (c) 2020, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Processing data for pretraining."""

import argparse
import json
import multiprocessing
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__),
                                             os.path.pardir)))
import time

import torch
try:
    import nltk
    nltk_available = True
except ImportError:
    nltk_available = False

from megatron.tokenizer import build_tokenizer
from megatron.data import indexed_dataset


# https://stackoverflow.com/questions/33139531/preserve-empty-lines-with-nltks-punkt-tokenizer
class CustomLanguageVars(nltk.tokenize.punkt.PunktLanguageVars):

    _period_context_fmt = r"""
        \S*                          # some word material
        %(SentEndChars)s             # a potential sentence ending
        \s*                       #  <-- THIS is what I changed
        (?=(?P<after_tok>
            %(NonWord)s              # either other punctuation
            |
            (?P<next_tok>\S+)     #  <-- Normally you would have \s+ here
        ))"""

class IdentitySplitter(object):
    def tokenize(self, *text):
        return text
"""[TODO]: modify this class to integrate the custom sentence splitter above """

class Encoder(object):
    def __init__(self, args):
        self.args = args
    
    def initializer(self):
        # Use Encoder class as a container for global data
        Encoder.tokenizer = build_tokenizer(self.args)
        if self.args.split_sentences:
            if not nltk_available:
                print("NLTK is not available to split sentences.")
                exit()
            splitter = nltk.load("tokenizers/punkt/english.pickle")
            if self.args.keep_newlines:
                # this prevents punkt from eating newlines after sentences
                Encoder.splitter = nltk.tokenize.punkt.PunktSentenceTokenizer(
                    train_text = splitter._params,
                    lang_vars = CustomLanguageVars())
            else:
                Encoder.splitter = splitter

        else:
            Encoder.splitter = IdentitySplitter()

    def encode(self, json_line):
        data = json.loads(json_line)
        ids = {}
        for key in self.args.json_keys:
            text = data[key]
            doc_ids = []
            for sentence in Encoder.splitter.tokenize(text):
                sentence_ids = Encoder.tokenizer.tokenize(sentence)
                if len(sentence_ids) > 0:
                    doc_ids.append(sentence_ids)
            if len(doc_ids) > 0 and self.args.append_eod:
                doc_ids[-1].append(Encoder.tokenizer.eod)
            ids[key] = doc_ids
        return ids, len(json_line)

def get_args():
    parser = argparse.ArgumentParser()
    group = parser.add_argument_group(title='input data')
    group.add_argument('--input', type=str, required=True,
                       help='Path to input JSON')
    group.add_argument('--json-keys', nargs='+', default=['text'],
                       help='space separate listed of keys to extract from json')
    group.add_argument('--split-sentences', action='store_true',
                       help='Split documents into sentences.')
    group.add_argument('--keep-newlines', action='store_true',
                       help='Keep newlines between sentences when splitting.')

    group = parser.add_argument_group(title='tokenizer')
    group.add_argument('--tokenizer-type', type=str, required=True,
                       choices=['BertWordPieceLowerCase','BertWordPieceCase',
                                'GPT2BPETokenizer'],
                       help='What type of tokenizer to use.')
    group.add_argument('--vocab-file', type=str, default=None,
                       help='Path to the vocab file')
    group.add_argument('--merge-file', type=str, default=None,
                       help='Path to the BPE merge file (if necessary).')
    group.add_argument('--append-eod', action='store_true',
                       help='Append an <eod> token to the end of a document.')


    group = parser.add_argument_group(title='output data')
    group.add_argument('--output-prefix', type=str, required=True,
                       help='Path to binary output file without suffix')
    group.add_argument('--dataset-impl', type=str, default='mmap',
                       choices=['lazy', 'cached', 'mmap'])

    group = parser.add_argument_group(title='runtime')
    group.add_argument('--workers', type=int, default=1,
                       help='Number of worker processes to launch')
    group.add_argument('--log-interval', type=int, default=100,
                       help='Interval between progress updates')
    args = parser.parse_args()
    args.keep_empty = False

    if args.tokenizer_type.lower().startswith('bert'):
        if not args.split_sentences:
            print("Bert tokenizer detected, are you sure you don't want to split sentences?")

    # some default/dummy values for the tokenizer
    args.rank = 0
    args.make_vocab_size_divisible_by = 128
    args.tensor_model_parallel_size = 1
    args.vocab_extra_ids = 0

    return args

def main():
    args = get_args()
    startup_start = time.time()

    print("Opening", args.input)
    fin = open(args.input, 'r', encoding='utf-8')

    if nltk_available and args.split_sentences:
        nltk.download("punkt", quiet=True)

    encoder = Encoder(args)
    tokenizer = build_tokenizer(args)
    pool = multiprocessing.Pool(args.workers, initializer=encoder.initializer)
    encoded_docs = pool.imap(encoder.encode, fin, 25)
    #encoded_docs = map(encoder.encode, fin)

    level = "document"
    if args.split_sentences:
        level = "sentence"

    print(f"Vocab size: {tokenizer.vocab_size}")
    print(f"Output prefix: {args.output_prefix}")
    output_bin_files = {}
    output_idx_files = {}
    builders = {}
    for key in args.json_keys:
        output_bin_files[key] = "{}_{}_{}.bin".format(args.output_prefix,
                                                      key, level)
        output_idx_files[key] = "{}_{}_{}.idx".format(args.output_prefix,
                                                      key, level)
        builders[key] = indexed_dataset.make_builder(output_bin_files[key],
                                               impl=args.dataset_impl,
                                               vocab_size=tokenizer.vocab_size)

    startup_end = time.time()
    proc_start = time.time()
    total_bytes_processed = 0
    print("Time to startup:", startup_end - startup_start)

    for i, (doc, bytes_processed) in enumerate(encoded_docs, start=1):
        total_bytes_processed += bytes_processed
        for key, sentences in doc.items():
            if len(sentences) == 0:
                continue
            for sentence in sentences:
                builders[key].add_item(torch.IntTensor(sentence))
            builders[key].end_document()
        if i % args.log_interval == 0:
            current = time.time()
            elapsed = current - proc_start
            mbs = total_bytes_processed/elapsed/1024/1024
            print(f"Processed {i} documents",
                  f"({i/elapsed} docs/s, {mbs} MB/s).",
                  file=sys.stderr)

    for key in args.json_keys:
        builders[key].finalize(output_idx_files[key])

if __name__ == '__main__':
    main()

Below cell block specify all the input parameters in order to run `MYpreprocess_data.py`. 

Please do **NOT** modify anything in below cell.

In [ ]:
INPUT_JSON_FILE='../dataset/SV/webnyheter2013.json'
OUTPUT_PATH='../dataset/SV/customSentenceSplit'
VOCAB_FILE='../dataset/SV/32k/vocab.json'
MERGE_FILE='../dataset/SV/32k/merges.txt'
NUM_CPUS=16

Below code block is a ReRun cell to launch `MYpreprocess_data.py` and produce the customSentenceSplit_text_document.bin and customSentenceSplit_text_document.idx files, if the script runs successfully.

<a id="Rerun_Cell"></a>

Go back and modify `MYpreprocess_data.py`, click on this shortcut link to <a href="./Lab2-4_customize_process2mmap.ipynb#MODIFY_CELL">Jump to Modify MYpreprocess_data.py</a> 

In [ ]:
!python ./Megatron-LM/tools/MYpreprocess_data.py \
                       --input $INPUT_JSON_FILE \
                       --output-prefix $OUTPUT_PATH \
                       --json-keys text \
                       --vocab-file $VOCAB_FILE \
                       --merge-file $MERGE_FILE \
                       --dataset-impl mmap \
                       --tokenizer-type GPT2BPETokenizer \
                       --workers $NUM_CPUS \
                       --append-eod

Check whether these two files : `customSentenceSplit_text_document.bin` and `customSentenceSplit_text_document.idx` files were successfully generated and is in the correct folder under dataset.

In [ ]:
! ls ../dataset/SV/

In [ ]:
## clean up to free up space
!rm ./Megatron-LM/tools/MYpreprocess_data.py

-----
## <p style="text-align:center;border:3px; padding: 1em"> <a href=../Start_Here.ipynb>HOME</a> &nbsp; &nbsp; &nbsp; <a href=./Lab2-5_run_Megatron_with_varying_config.ipynb>NEXT</a></p>

-----


## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). 